In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-ml-w6p2/submit.csv
/kaggle/input/2024-ml-w6p2/train.csv
/kaggle/input/2024-ml-w6p2/test.csv


In [2]:
#데이터 로드
train = pd.read_csv('/kaggle/input/2024-ml-w6p2/train.csv')
test = pd.read_csv('/kaggle/input/2024-ml-w6p2/test.csv')
submit = pd.read_csv('/kaggle/input/2024-ml-w6p2/submit.csv')

In [3]:
#랜덤시드 고정
import random
import os

seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [4]:
#train 데이터 확인
train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diabetes
0,1,95,60,18,58,23.9,0.260,22,0
1,5,105,72,29,325,36.9,0.159,28,0
2,0,135,68,42,250,42.3,0.365,24,1
3,4,131,68,21,166,33.1,0.160,28,0
4,1,103,30,38,83,43.3,0.183,33,0


In [5]:
#데이터 분할
x = train.drop('Diabetes', axis=1)
y = train['Diabetes']

from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=seed, stratify=y)

In [6]:
#정규화
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x_train_std = std.fit_transform(x_train)
x_test_std = std.transform(x_test)

In [7]:
#QDA 학습
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda=QuadraticDiscriminantAnalysis()

qda.fit(x_train_std, y_train) # QDA 적합
y_train_pred=qda.predict(x_train_std)
y_test_pred=qda.predict(x_test_std)

In [8]:
#분류 성능
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))

#분류 결과
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))

0.7626666666666667
0.808641975308642
[[96  9]
 [22 35]]


In [9]:
#LDA 학습
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
x_lda = lda.fit_transform(x_train_std, y_train) 

#예측
y_train_pred = lda.predict(x_train_std)
y_test_pred = lda.predict(x_test_std)

In [10]:
#분류 성능
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))

#분류 결과
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))

0.7706666666666667
0.8024691358024691
[[98  7]
 [25 32]]


In [11]:
#전체 테스트 데이터에 대해 학습
qda.fit(x, y)
test_pred = qda.predict(test)

In [12]:
#제출
submit['Diabetes'] = test_pred
submit.to_csv('submit.csv', index=False)